# 1. Gestión del dataset - Estratificación automática + RECORTE

In [2]:
import cv2
import pandas as pd
from tqdm import tqdm
import os
from sklearn.model_selection import StratifiedGroupKFold, train_test_split
from typing import Tuple
import numpy as np

# Función para obtener el bounding box (ya la tienes)
def obtain_bb(threshed_image : np.ndarray) -> Tuple[int, int, int, int]:
    contours, _ = cv2.findContours(threshed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        rect = cv2.boundingRect(c)
        if rect[2] < 100 or rect[3] < 100:
            continue
        else:
            x, y, w, h = rect
    return x, y, w, h

In [3]:
# Función para calcular la relación de intensidad (ya la tienes)
def intensity_ratio(gray_image : np.ndarray, x : int, y : int, w : int, h : int) -> float:
    cropped_img = gray_image[y:y+h, x:x+w]
    intensity = np.sum(cropped_img)  # Suma de los valores de intensidad
    cii = intensity / (cropped_img.shape[0] * cropped_img.shape[1])

    intensity_f = np.sum(gray_image)  # Suma de los valores de intensidad para la imagen completa
    fii = intensity_f / (gray_image.shape[0] * gray_image.shape[1])
    
    return cii / fii

In [4]:
# Función para recortar imágenes
def crop_images(df: pd.DataFrame, img_dir: str, destination_dir: str):
    """
    Recorta las imágenes que contienen la lesión y las guarda en el directorio destino.
    """
    for i, row in tqdm(df.iterrows(), total=len(df)):
        img_path = os.path.join(img_dir, row['bcn_filename'])
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        
        if img is None:
            print(f"Error al leer la imagen: {img_path}")
            continue
        
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray_image, 60, 255, 0)
        
        try:
            x, y, w, h = obtain_bb(thresh)
            int_ratio = intensity_ratio(gray_image, x, y, w, h)
            
            if int_ratio > 1.1:  # Solo recortamos si la relación de intensidad es significativa
                cropped_img = img[y:y+h, x:x+w]
                
                # Guardamos la imagen recortada en el directorio destino
                destination_path = os.path.join(destination_dir, row['bcn_filename'])
                os.makedirs(os.path.dirname(destination_path), exist_ok=True)  # Asegura que el directorio exista
                cv2.imwrite(destination_path, cropped_img)
                
                # Actualizamos el DataFrame con la nueva ruta
                df.at[i, 'filepath'] = destination_path
        except Exception as e:
            print(f"No se pudo procesar la imagen {row['bcn_filename']}: {e}")

Cargar CSV original y visualizar los datos. Verificamos que las columnas clave como bcn_filename, diagnosis y lesion_id están presentes en el archivo CSV

In [5]:
# Cargar el CSV original
csv_path = r"C:\BCN20000\bcn_20k_train.csv"
df = pd.read_csv(csv_path)

# Mostramos las primeras filas para revisar la estructura
df.head()

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train


Añadir la ruta completa a cada imagen

In [6]:
# Directorios de origen y destino
img_dir = r"C:\BCN20000\BCN_20k_train\bcn_20k_train"
destination_dir = r"C:\BCN20000\cropped_images"  # Nuevo directorio para las imágenes recortadas

# Añadir las rutas completas de las imágenes al DataFrame
df["filepath"] = df["bcn_filename"].apply(lambda x: os.path.join(img_dir, x))

# Verificamos
df.head()

,bcn_filename,age_approx,anatom_site_general,diagnosis,lesion_id,capture_date,sex,split,filepath
0,BCN_0000000001.jpg,55.0,anterior torso,MEL,BCN_0003884,2012-05-16,male,train,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...
1,BCN_0000000003.jpg,50.0,anterior torso,MEL,BCN_0000019,2015-07-09,female,train,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...
2,BCN_0000000004.jpg,85.0,head/neck,SCC,BCN_0003499,2015-11-23,male,train,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...
3,BCN_0000000006.jpg,60.0,anterior torso,NV,BCN_0003316,2015-06-16,male,train,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...
4,BCN_0000000010.jpg,30.0,anterior torso,BCC,BCN_0004874,2014-02-18,female,train,C:\BCN20000\BCN_20k_train\bcn_20k_train\BCN_00...


Asignamos etiquetas numéricas a cada diagnóstico

In [7]:
# Llamamos a la función para recortar y guardar las imágenes
crop_images(df, img_dir, destination_dir)

100%|██████████| 12413/12413 [06:34<00:00, 31.49it/s]


In [8]:
# Mapear las etiquetas
label_map = {label: idx for idx, label in enumerate(sorted(df["diagnosis"].unique()))}
df["label"] = df["diagnosis"].map(label_map)

# Mostrar las etiquetas asignadas
print("Etiquetas asignadas:")
print(label_map)

Etiquetas asignadas:
{'AK': 0, 'BCC': 1, 'BKL': 2, 'DF': 3, 'MEL': 4, 'NV': 5, 'SCC': 6, 'VASC': 7}


Dividir de forma estratificada sin mezclar lesion_id

Usamos StratifiedGroupKFold para que no haya imágenes de la misma lesion_id en splits diferentes.

In [9]:
# Dividir en train/val/test usando StratifiedGroupKFold y train_test_split
sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

# Creamos los splits de train, val, y test
for train_val_idx, test_idx in sgkf.split(df, df["label"], groups=df["lesion_id"]):
    break  # Solo usamos un split para este ejemplo

train_val_df = df.iloc[train_val_idx]
test_df = df.iloc[test_idx]

Ahora hacemos otro split dentro del conjunto train_val_df para obtener validación

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.1,  # 10% para validación
    stratify=train_val_df["label"],
    random_state=42
)

# Mostramos tamaños
print(f"Train: {len(train_df)}")
print(f"Validation: {len(val_df)}")
print(f"Test: {len(test_df)}")

Splits guardados con éxito.
Train: 8971
Validation: 997
Test: 2445


Guardar CSVs para usar en TensorFlow

In [36]:
# Guardamos los splits como CSVs
train_df.to_csv("train_split.csv", index=False)
val_df.to_csv("val_split.csv", index=False)
test_df.to_csv("test_split.csv", index=False)

print("Splits guardados con éxito.")

Splits guardados con éxito.


Acabamos de crear tres archivos CSV (train_split.csv, val_split.csv, test_split.csv) que apuntan a las rutas absolutas de las imágenes, sin mover, copiar ni alterar nada en la carpeta original BCN20000.

Evitamos tocar el dataset original: la carpeta con imágenes mezcladas queda intacta.

Evitamos duplicados o inconsistencias: cada split es coherente y sin data leakage (gracias a la separación por lesion_id).

Compatibilidad con TensorFlow.